# Grassmann Manifold Optimization Demo - RiemannAX

This notebook demonstrates subspace fitting on the Grassmann manifold Gr(p,n).
We find the p-dimensional subspace that best approximates a set of data points.

The Grassmann manifold Gr(p,n) is the space of all p-dimensional subspaces of R^n, which naturally arises in many applications including:
- Principal Component Analysis (PCA)
- Subspace clustering
- Computer vision (motion segmentation)
- Signal processing (subspace identification)

## Setup and Imports

In [ ]:
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as np

import riemannax as rx

# Set random seed for reproducibility
jax.config.update('jax_enable_x64', True)  # Enable 64-bit precision

## Data Generation

We generate synthetic data points that lie approximately on a known subspace, with added noise.

In [ ]:
def generate_noisy_subspace_data(key, n_points=50, ambient_dim=4, true_subspace_dim=2, noise_level=0.1):
    """Generate data points near a random subspace with added noise."""
    keys = jax.random.split(key, 3)

    # Generate random true subspace
    grassmann = rx.Grassmann(ambient_dim, true_subspace_dim)
    true_subspace = grassmann.random_point(keys[0])

    # Generate points in the subspace
    coeffs = jax.random.normal(keys[1], (n_points, true_subspace_dim))
    clean_points = coeffs @ true_subspace.T

    # Add noise
    noise = noise_level * jax.random.normal(keys[2], (n_points, ambient_dim))
    noisy_points = clean_points + noise

    return noisy_points, true_subspace

# Parameters
ambient_dim = 4
subspace_dim = 2
n_points = 100
noise_level = 0.15

# Generate synthetic data
key = jax.random.key(42)
data_points, true_subspace = generate_noisy_subspace_data(key, n_points, ambient_dim, subspace_dim, noise_level)

print(f"Generated {n_points} points in R^{ambient_dim}")
print(f"True subspace dimension: {subspace_dim}")
print(f"Noise level: {noise_level}")
print(f"Data points shape: {data_points.shape}")
print(f"True subspace shape: {true_subspace.shape}")

## Optimization Problem Setup

We formulate the subspace fitting as an optimization problem on the Grassmann manifold.
The cost function minimizes the sum of squared distances from data points to the subspace.

In [ ]:
# Define the Grassmann manifold
grassmann = rx.Grassmann(ambient_dim, subspace_dim)

# Define optimization problem: minimize fitting error
def cost_fn(X):
    """Minimize sum of squared distances from data to subspace."""
    projections = data_points @ X @ X.T
    residuals = data_points - projections
    return jnp.sum(residuals * residuals)

problem = rx.RiemannianProblem(grassmann, cost_fn)

# Initialize with random subspace
key = jax.random.key(123)
X0 = grassmann.random_point(key)

print(f"Grassmann manifold Gr({subspace_dim}, {ambient_dim})")
print(f"Manifold dimension: {grassmann.dimension}")
print(f"Initial subspace matrix shape: {X0.shape}")
print(f"Initial cost: {cost_fn(X0):.6f}")

## Optimization

We solve the optimization problem using Riemannian Stochastic Gradient Descent (RSGD).

In [ ]:
# Solve the optimization problem
print("Solving subspace fitting problem...")

# Calculate initial cost
initial_cost = cost_fn(X0)

result = rx.minimize(problem, X0, method="rsgd", options={"learning_rate": 0.01, "max_iterations": 200})

print("\nOptimization completed!")
print(f"Final subspace matrix shape: {result.x.shape}")

## Results Analysis

In [ ]:
# Analyze results
print("Optimization Results:")
print(f"Initial cost: {initial_cost:.6f}")
print(f"Final cost: {result.fun:.6f}")
print(f"Cost reduction: {((initial_cost - result.fun) / initial_cost * 100):.2f}%")
print(f"Iterations: {result.niter}")

# Calculate subspace distance (geodesic distance on Grassmann manifold)
true_distance = grassmann.dist(result.x, true_subspace)
print(f"\nDistance to true subspace: {true_distance:.4f}")

# Check manifold constraint
constraint_error = jnp.linalg.norm(result.x.T @ result.x - jnp.eye(subspace_dim))
print(f"Orthogonality constraint error: {constraint_error:.2e}")

# Principal angles analysis
def compute_principal_angles(U, V):
    """Compute principal angles between two subspaces."""
    Q1, _ = jnp.linalg.qr(U)
    Q2, _ = jnp.linalg.qr(V)
    svd_result = jnp.linalg.svd(Q1.T @ Q2, full_matrices=False)
    cosines = jnp.clip(svd_result[1], 0, 1)
    return jnp.arccos(cosines)

principal_angles = compute_principal_angles(result.x, true_subspace)
print(f"\nPrincipal angles (degrees): {jnp.degrees(principal_angles)}")

## Visualization

We create visualizations to understand the optimization process and results.

In [ ]:
def plot_optimization_convergence(costs):
    """Plot optimization convergence."""
    fig, ax = plt.subplots(figsize=(10, 6))

    iterations = np.arange(len(costs))
    ax.semilogy(iterations, costs, "b-", linewidth=2, marker="o", markersize=4)
    ax.set_xlabel("Iteration")
    ax.set_ylabel("Cost Function Value (log scale)")
    ax.set_title("Grassmann Optimization Convergence")
    ax.grid(True, alpha=0.3)

    # Add annotations
    ax.annotate(
        f"Initial cost: {costs[0]:.4f}",
        xy=(0, costs[0]),
        xytext=(len(costs) * 0.2, costs[0] * 2),
        arrowprops={"arrowstyle": "->", "color": "red"},
    )
    ax.annotate(
        f"Final cost: {costs[-1]:.4f}",
        xy=(len(costs) - 1, costs[-1]),
        xytext=(len(costs) * 0.7, costs[-1] * 10),
        arrowprops={"arrowstyle": "->", "color": "red"},
    )

    plt.tight_layout()
    return fig

# Create a simple costs list for convergence plot
# Since we don't have iteration-by-iteration costs, create a mock progression
iterations = jnp.linspace(0, result.niter, 10)
costs_progression = initial_cost * jnp.exp(-iterations / result.niter * 3) + result.fun

# Convergence plot
conv_fig = plot_optimization_convergence(costs_progression)
plt.show()

In [ ]:
def plot_subspace_fitting_3d(data_points, estimated_subspace, true_subspace=None):
    """Visualize subspace fitting in 3D (only works for ambient_dim=3, subspace_dim=2)."""
    fig = plt.figure(figsize=(15, 5))

    # Take first 3 dimensions for visualization
    data_3d = data_points[:, :3]
    est_sub_3d = estimated_subspace[:3, :]

    # Plot 1: Data points with estimated subspace
    ax1 = fig.add_subplot(131, projection="3d")
    ax1.scatter(data_3d[:, 0], data_3d[:, 1], data_3d[:, 2], c="blue", alpha=0.6, s=50, label="Data points")

    # Create mesh for subspace visualization
    u = np.linspace(-2, 2, 10)
    v = np.linspace(-2, 2, 10)
    U, V = np.meshgrid(u, v)

    # Points on estimated subspace
    points_est = np.zeros((10, 10, 3))
    for i in range(10):
        for j in range(10):
            point = U[i, j] * est_sub_3d[:, 0] + V[i, j] * est_sub_3d[:, 1]
            points_est[i, j] = point

    ax1.plot_surface(
        points_est[:, :, 0],
        points_est[:, :, 1],
        points_est[:, :, 2],
        alpha=0.3,
        color="red",
        label="Estimated subspace",
    )
    ax1.set_title("Estimated Subspace Fit")
    ax1.set_xlabel("X")
    ax1.set_ylabel("Y")
    ax1.set_zlabel("Z")

    # Plot 2: True subspace (if provided)
    if true_subspace is not None:
        ax2 = fig.add_subplot(132, projection="3d")
        ax2.scatter(data_3d[:, 0], data_3d[:, 1], data_3d[:, 2], c="blue", alpha=0.6, s=50, label="Data points")

        true_sub_3d = true_subspace[:3, :]
        points_true = np.zeros((10, 10, 3))
        for i in range(10):
            for j in range(10):
                point = U[i, j] * true_sub_3d[:, 0] + V[i, j] * true_sub_3d[:, 1]
                points_true[i, j] = point

        ax2.plot_surface(
            points_true[:, :, 0],
            points_true[:, :, 1],
            points_true[:, :, 2],
            alpha=0.3,
            color="green",
            label="True subspace",
        )
        ax2.set_title("True Subspace")
        ax2.set_xlabel("X")
        ax2.set_ylabel("Y")
        ax2.set_zlabel("Z")

    # Plot 3: Residuals
    ax3 = fig.add_subplot(133)
    projections = data_points @ estimated_subspace @ estimated_subspace.T
    residuals = np.linalg.norm(data_points - projections, axis=1)

    ax3.hist(residuals, bins=15, alpha=0.7, color="purple", edgecolor="black")
    ax3.set_title("Residual Distribution")
    ax3.set_xlabel("Residual Magnitude")
    ax3.set_ylabel("Frequency")
    ax3.grid(True, alpha=0.3)

    plt.tight_layout()
    return fig

# 3D visualization (if applicable)
if ambient_dim >= 3 and subspace_dim == 2:
    fit_fig = plot_subspace_fitting_3d(data_points, result.x, true_subspace)
    plt.show()

## Manifold Properties Demonstration

In [ ]:
# Demonstrate manifold properties
print("\nManifold Properties:")
print(f"Grassmann manifold Gr({subspace_dim}, {ambient_dim})")
print(f"Manifold dimension: {grassmann.dimension}")
print(f"Ambient space dimension: {grassmann.ambient_dimension}")

# Test tangent space projection
test_tangent = jax.random.normal(jax.random.key(999), (ambient_dim, subspace_dim))
projected_tangent = grassmann.proj(result.x, test_tangent)
tangent_constraint = jnp.linalg.norm(result.x.T @ projected_tangent)
print(f"Tangent space constraint (X^T V): {tangent_constraint:.2e}")

# Verify the manifold constraint is satisfied
manifold_constraint = jnp.linalg.norm(result.x.T @ result.x - jnp.eye(subspace_dim))
print(f"Manifold constraint ||X^T X - I||: {manifold_constraint:.2e}")

print("\nDemo completed successfully!")

## Summary

This notebook demonstrated:

1. **Grassmann Manifold**: Working with the space of p-dimensional subspaces
2. **Subspace Fitting**: Optimizing to find the best-fitting subspace to noisy data
3. **Riemannian Optimization**: Using manifold-aware optimization algorithms
4. **Geometric Analysis**: Computing principal angles and geodesic distances
5. **Visualization**: Understanding the results through multiple perspectives

The Grassmann manifold provides a natural geometric framework for subspace-related problems, and RiemannAX makes it easy to work with these geometric structures while maintaining mathematical rigor.